In [4]:
%matplotlib widget

import pickle
from IPython import display
from base64 import b64decode
from pacti.terms.polyhedra import *
from pacti.utils import read_contracts_from_file
from contract_utils import *

In [7]:

f5 = open('data/results5.data', 'rb')
results5 = pickle.load(f5)
f5.close()

f20 = open('data/results20.data', 'rb')
results20 = pickle.load(f20)
f20.close()
